<a href="https://colab.research.google.com/github/LoveMachineX/CSC_496_final_project/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1w1rNOGCfM4wNO1KASnvx6FBHwxiODOh_

Downloading...
From: https://drive.google.com/uc?id=1w1rNOGCfM4wNO1KASnvx6FBHwxiODOh_
To: /content/affective_polarization_train.csv
100% 1.46M/1.46M [00:00<00:00, 98.5MB/s]


In [ ]:
# load data
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import re

df = pd.read_csv('affective_polarization_train.csv')

In [ ]:
df.head(10)

,text,labels,screen_name,twitter_lower,party
0,BREAKING: Congress is voting on a new package ...,0.0,ABrindisiNY,abrindisiny,D
1,Budget highlights for me include @NYGovCuomo a...,0.0,ABrindisiNY,abrindisiny,D
2,Republicans' War on Women has now officially r...,1.0,AGBecerra,agbecerra,D
3,"Thanks to the #ACA, Californians now have affo...",0.0,AGBecerra,agbecerra,D
4,@BetsyDeVosED Unscrupulous #forprofit purveyor...,0.0,AGBecerra,agbecerra,D
5,We secured a dismissal of a lawsuit seeking to...,0.0,AGBecerra,agbecerra,D
6,Hard working Americans saving for #retirement ...,0.0,AGBecerra,agbecerra,D
7,RT if you agree: Congress should stay in sessi...,0.0,AGBecerra,agbecerra,D
8,If you’re just as sick of gun violence as an A...,0.0,AOC,aoc,D
9,Why do we need to end cash bail? @CabanForQue...,0.0,AOC,aoc,D


preprocess the text data


In [ ]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import nltk
import re

# Download NLTK data for tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

df['text'] = df['text'].astype(str)

# Preprocessing function
def preprocess_text(text):
    # Remove URLs, mentions, and special characters
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    # Lowercase the text
    text = text.lower()
    return text

# Apply preprocessing
df['processed_text'] = df['text'].apply(preprocess_text)
df['tokens'] = df['processed_text'].apply(lambda x: x.split())



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Prepare tokenized sentences for Word2Vec
tokenized_sentences = df['tokens'].tolist()

In [ ]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=2, sg=1, epochs=20)

In [ ]:
# Make classification
# Calculate the average Word2Vec embeddings for each tweet
def get_average_embedding(tokens, model):
    valid_tokens = [token for token in tokens if token in model.wv.key_to_index]
    if valid_tokens:
        return np.mean(model.wv[valid_tokens], axis=0)
    else:
        return np.zeros(model.vector_size)



In [ ]:
# Generate features (average embeddings) for all tweets
df['embedding'] = df['tokens'].apply(lambda tokens: get_average_embedding(tokens, word2vec_model))
X = np.stack(df['embedding'].values)  # Features
y = df['labels'].fillna(0).astype(int)  # Target labels, replacing NaN with 0

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Train a Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Make predictions
y_pred = classifier.predict(X_test)

In [ ]:
# Evaluate the model
report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])
report

'              precision    recall  f1-score   support\n\n     Class 0       0.94      0.99      0.96      1589\n     Class 1       0.73      0.38      0.50       157\n\n    accuracy                           0.93      1746\n   macro avg       0.84      0.68      0.73      1746\nweighted avg       0.92      0.93      0.92      1746\n'

In [ ]:
# Find similar words to terms of interest
terms_of_interest = ["health", "covid19", "pandemic", "public"]
similar_words = {term: word2vec_model.wv.most_similar(term, topn=10) for term in terms_of_interest if term in word2vec_model.wv.key_to_index}

In [ ]:
similar_words

{'health': [('care', 0.8392959833145142),
  ('mental', 0.6205944418907166),
  ('tens', 0.6185731291770935),
  ('affordable', 0.5973840355873108),
  ('americâ', 0.5920407772064209),
  ('professionals', 0.5905299782752991),
  ('rationed', 0.5888785719871521),
  ('denials', 0.5871626734733582),
  ('throwing', 0.5771002769470215),
  ('hijacking', 0.5762830972671509)],
 'covid19': [('surges', 0.6724194288253784),
  ('dissatisfied', 0.66228848695755),
  ('wasco', 0.6612771153450012),
  ('itend', 0.6339424848556519),
  ('loads', 0.6294421553611755),
  ('impacting', 0.6247754096984863),
  ('dealing', 0.6164299249649048),
  ('wide', 0.6137675642967224),
  ('distribution', 0.6081402897834778),
  ('fronteriza', 0.6067398190498352)],
 'pandemic': [('frontlines', 0.694251537322998),
  ('plaguing', 0.6908631920814514),
  ('undocumented', 0.6888308525085449),
  ('workplaces', 0.6751034259796143),
  ('warping', 0.6730729937553406),
  ('waterways', 0.6700007319450378),
  ('womenãââãâãâs', 0.66682893037